In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

remove_fight_island = True    #Removes fight island fights from consideration

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/upcoming-event.csv')
len(df)





4473

In [6]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_no_fight_island = df[(df['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    #df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df = df_no_fight_island
    #df_train = df_train_no_fight_island

len(df)

4354

In [7]:
num_upcoming_fights = len(df_upcoming)

In [8]:
df = df_upcoming.append(df)
len(df)

4366

In [9]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Matt-Predator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [11]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [12]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4354
4354
4354


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\Matt-Predator\Anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Paul Felder,Rafael Dos Anjos,165,-200,165.000000,50.000000,2020-11-14,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Abdul Razak Alhassan,Khaos Williams,-240,185,41.666667,185.000000,2020-11-14,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,Kay Hansen,Cory McKenna,-230,180,43.478261,180.000000,2020-11-14,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,Brendan Allen,Sean Strickland,-118,-106,84.745763,94.339623,2020-11-14,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,Ashley Yoder,Miranda Granger,135,-167,135.000000,59.880240,2020-11-14,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4469,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4470,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4471,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Deiveson Figueiredo,Alex Perez,-286,225,34.965035,225.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Valentina Shevchenko,Jennifer Maia,-1667,850,5.998800,850.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mike Perry,Tim Means,-150,120,66.666667,120.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Katlyn Chookagian,Cynthia Calvillo,205,-265,205.000000,37.735849,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mauricio Rua,Paul Craig,150,-190,150.000000,52.631579,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Brandon Moreno,Brandon Royval,-190,150,52.631579,150.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Joaquin Buckley,Jordan Wright,-265,205,37.735849,205.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Antonina Shevchenko,Ariane Lipski,-167,130,59.880240,130.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Daniel Rodriguez,Nicolas Dalby,-335,250,29.850746,250.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Alan Jouban,Jared Gooden,-150,120,66.666667,120.000000,2020-11-21,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [16]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    

    return(0)

In [17]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice:
model_choice = 13

print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20', 'model_9_2_20', 'model_9_10_20', 'model_10_23_20', 'model_10_27_20', 'model_10_28_20'], ['LogisticRegression(max_iter=50000, random_state=75)', "DecisionTreeClassifier(max_depth=27, max_leaf_nodes=336, min_samples_leaf=3,\n                       random_state=75, splitter='random')", "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=22,\n                       n_jobs=-1, random_state=75)", "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=6,\n                           n_estimators=195, random_state=75)", 'GaussianNB(var_smoothing=1e-08)', 'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'GaussianNB(var_smoo

In [18]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Deiveson Figueiredo vs. Alex Perez
No bets.  EV1:-0.7647899384471657. EV2: -13.96147086065639
Deiveson Figueiredo has an EV of -0.76 on odds of -286.  They have 73.53 % chance of winning.

Valentina Shevchenko vs. Jennifer Maia
Jennifer Maia has an EV of 16.54 on odds of 850.  They have 12.27 % chance of winning.

Mike Perry vs. Tim Means
Tim Means has an EV of 9.62 on odds of 120.  They have 49.83 % chance of winning.

Katlyn Chookagian vs. Cynthia Calvillo
Katlyn Chookagian has an EV of 38.85 on odds of 205.  They have 45.53 % chance of winning.

Mauricio Rua vs. Paul Craig
Mauricio Rua has an EV of 26.29 on odds of 150.  They have 50.52 % chance of winning.

Brandon Moreno vs. Brandon Royval
Brandon Royval has an EV of 5.82 on odds of 150.  They have 42.33 % chance of winning.

Joaquin Buckley vs. Jordan Wright
No bets.  EV1:-2.7508267944624016. EV2: -10.346970529796614
Joaquin Buckley has an EV of -2.75 on odds of -265.  They have 70.61 % chance of winning.

Antonina Shevchenko vs.

0